## Day 4

### Part 1
https://adventofcode.com/2024/day/4


So what is going on here?
- count the number of times the word 'XMAS' appears in the input

### Approach
- read in all the lines into a list
- find every single 'X' in the input and record its location
- create a function that, given the coordinates of some 'X', checks for all possible 'XMAS' occurrences, in all directions.
    - left and right
    - above and below
    - four diagonals


In [13]:
int count_XMAS(List<string> ws){
    // iterate through the wordSearch. Every time you find an X, inc result by check_X
    int ret = 0;
    for (int row = 0; row < ws.Count; row++){
        for (int col = 0; col < ws[row].Length; col++){
            if (ws[row][col] == 'X') { ret += check_X(ws, row, col); }
        }
    }
    
    return ret;
}

int check_X(List<string> ws, int row, int col){
    // given the column and row relating to an X,
    // check for all 8 possible occurrences of XMAS that contain it.

    string word = "XMAS"; // check the word using modulus i guess
    int word_pt = 0;
    int ret = 0;

    // left: check X...MAS
    word_pt = 0;
    for (int i = col; i < ws[row].Length; i++){
        if ( word_pt > 3 || ws[row][i] != word[word_pt]) { break; }
        else { word_pt += 1; }
    }
    if (word_pt > 3) {ret += 1;}
    
    return ret;

}

In [14]:
// Reads in input into a list
// format: list[row][col]
using System.IO;
List<string> wordSearch = new List<string>();

foreach (string line in File.ReadLines("day4-ex-input.txt")){
    // for each line, simply add to the wordSearch
    wordSearch.Add(line);
}

foreach (string s in wordSearch){
    Console.WriteLine(s);
}

int r = count_XMAS(wordSearch);
Console.WriteLine(r);

..X...
.SAMX.
.AXMAS
XMAS.S
.X....
2


## Part 2

We change what qualifies as a bad report: We can now tolerate a single bad level.

What this means:
* any report where removing a single level would make it safe also counts as safe.

How to handle the whole dampening thing:
* use a 0 and 1 index tracker
* when working normally, advance both
* when encountering the first failure, increment 1 tracker but not 0.
* when encountering the second failure, return false as before.

Current issue:
* getting the wrong number. I think it's because i always try to skip the deep number when i find an issue, but really i could also skip the deep number too.
* try a list method, i guess. Use recursion with both cut lists and return an OR.

Still not working
* next thing would be to try it exhaustively, removing every value.
* O(n^2)

In [22]:
// Checks if a report is safe (returns true) or unsafe (returns false)
// Has dampening, which means that any report where removing a single level would make it safe also counts as safe.
bool Validate_Report_WithDampening(List<int> report, bool canTolerate){
    if (report.Count == 1) return true;

    bool ascending = true;
    if (report[0] > report[1]) ascending = false;

    int deep = 1;
    int shallow = 0;
    while (deep < report.Count){
        int dif = report[deep] - report[shallow];
        // Console.WriteLine(report[shallow]);
        // Console.WriteLine(report[deep]);
        // Console.WriteLine(dif);

        // when ascending, [i] > [i-1], i.e. dif must be positive
        // when descending, [i] < [i-1], i.e. dif must be negative
        if ((ascending && dif > 0 && dif < 4) || (!ascending && dif < 0 && dif > -4)){
            shallow += 1;
            deep += 1;
        }
        else if (canTolerate){
            canTolerate = false;
            // try both subsections of the list and return their OR.
            List<int> cutShallow = new List<int>(report);
            List<int> cutDeep = new List<int>(report);

            if (deep + 1 < report.Count) cutShallow.RemoveAt(deep + 1);
            cutDeep.RemoveAt(shallow);

            return Validate_Report_WithDampening(cutShallow, canTolerate) 
                || Validate_Report_WithDampening(cutDeep, canTolerate);

        }
        else return false;
    }
    return true;
}

In [20]:
List<int> testList1 = new List<int>(){7,6,4,2,1};
List<int> testList2 = new List<int>(){1,2,7,8,9};
List<int> testList3 = new List<int>(){9,7,6,2,1};
List<int> testList4 = new List<int>(){1,3,2,4,5};
List<int> testList5 = new List<int>(){8,6,4,4,1};
List<int> testList6 = new List<int>(){1,3,6,7,9};

bool result1 = Validate_Report_WithDampening(testList1, true);
bool result2 = Validate_Report_WithDampening(testList2, true);
bool result3 = Validate_Report_WithDampening(testList3, true);
bool result4 = Validate_Report_WithDampening(testList4, true);
bool result5 = Validate_Report_WithDampening(testList5, true);
bool result6 = Validate_Report_WithDampening(testList6, true);

Console.WriteLine(result1);
Console.WriteLine(result2);
Console.WriteLine(result3);
Console.WriteLine(result4);
Console.WriteLine(result5);
Console.WriteLine(result6);

True
False
False
True
True
True


In [21]:
using System.IO;
int safeSum = 0;
foreach (string line in File.ReadLines("day2-input.txt")){
    // int p1 = Convert.ToInt32(line.Substring(0, 5));
    // int p2 = Convert.ToInt32(line.Substring(8, 5));
    
    // split line to report.
    string[] reportStrs = line.Split(' ');
    // Console.WriteLine(reportStrs);

    // convert strings to ints.
    int[] reportInts = Array.ConvertAll(reportStrs, int.Parse);
    List<int> reportIntsList = reportInts.ToList();
    // Console.WriteLine(reportInts);
    // Console.WriteLine(reportIntsList);
    
    // Check safety and increase safeSum.
    if (Validate_Report_WithDampening(reportIntsList, true)) safeSum += 1;
}

Console.WriteLine((safeSum));

587
